In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import (AdaBoostClassifier, AdaBoostRegressor,
                              BaggingClassifier, GradientBoostingClassifier,
                              GradientBoostingRegressor,
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC, LinearSVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:.2f}'.format


In [2]:
# ファイル読み込み
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submit = pd.read_csv('sample_submit.csv', header=None)


In [3]:
train.head(1)


,id,賃料,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間
0,1,75000,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,9年9ヶ月,南東,20.01m2,1階／12階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,2年間


In [4]:
train.columns


Index(['id', '賃料', '所在地', 'アクセス', '間取り', '築年数', '方角', '面積', '所在階', 'バス・トイレ',
       'キッチン', '放送・通信', '室内設備', '駐車場', '周辺環境', '建物構造', '契約期間'],
      dtype='object')

In [5]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31470 entries, 0 to 31469
Data columns (total 17 columns):
id        31470 non-null int64
賃料        31470 non-null int64
所在地       31470 non-null object
アクセス      31470 non-null object
間取り       31470 non-null object
築年数       31470 non-null object
方角        28721 non-null object
面積        31470 non-null object
所在階       31470 non-null object
バス・トイレ    31141 non-null object
キッチン      30849 non-null object
放送・通信     28030 non-null object
室内設備      31388 non-null object
駐車場       26183 non-null object
周辺環境      22038 non-null object
建物構造      31470 non-null object
契約期間      24350 non-null object
dtypes: int64(2), object(15)
memory usage: 4.1+ MB


In [6]:
train.isnull().sum()


id           0
賃料           0
所在地          0
アクセス         0
間取り          0
築年数          0
方角        2749
面積           0
所在階          0
バス・トイレ     329
キッチン       621
放送・通信     3440
室内設備        82
駐車場       5287
周辺環境      9432
建物構造         0
契約期間      7120
dtype: int64

In [7]:
train.describe(include='all')


,id,賃料,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間
count,31470.00,31470.00,31470,31470,31470,31470,28721,31470,31470,31141,30849,28030,31388,26183,22038,31470,24350
unique,nan,nan,11333,24172,30,746,8,5739,1032,360,381,139,6220,3911,17864,10,66
top,nan,nan,東京都板橋区板橋３丁目1-8,都営大江戸線\t新江古田駅\t徒歩10分\t\t西武池袋線\t江古田駅\t徒歩20分\t\t...,1K,0年1ヶ月,南,20m2,2階／2階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t24時間換気システム\...,駐車場\t無,【スーパー】 264m\t【スーパー】 848m\t【スーパー】 969m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間
freq,nan,nan,54,65,11930,983,8909,226,2851,4606,6016,2618,1540,3269,48,17505,22555
mean,15735.50,118290.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,9084.75,73748.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.00,25000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,7868.25,76000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,15735.50,102000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,23602.75,136000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# 所在地
train['所在地'] = train['所在地'].apply(lambda x: x.replace("丁目-", "丁目"))

train['都道府県'] = train['所在地'].apply(
    lambda x: x.split("都")[0]+"都" if "都" in x else pd.np.nan)
train['市区町村'] = train['所在地'].apply(lambda x: x.split(
    "都")[1].split("区")[0]+"区" if "区" in x else pd.np.nan)
train['丁目番地号'] = train['所在地'].apply(lambda x: x.split(
    "区")[1].split("丁目")[0]+"丁目" if "丁目" in x else pd.np.nan)
train['マンション名・部屋番号等'] = train['所在地'].apply(
    lambda x: x.split("丁目")[1] if "丁目" in x and x.split("丁目")[1] != "" else pd.np.nan)


In [9]:
# アクセス
train['鉄道会社'] = train['アクセス'].apply(lambda x: np.array(
    [z for z in [y.split("\t") for y in x.split("\t\t")] if len(z) == 3]).T[0])
train['駅名'] = train['アクセス'].apply(lambda x: np.array(
    [z for z in [y.split("\t") for y in x.split("\t\t")] if len(z) == 3]).T[1])
train['所要時間'] = train['アクセス'].apply(lambda x: np.array(
    [z for z in [y.split("\t") for y in x.split("\t\t")] if len(z) == 3]).T[2])

train['アクセス'] = train['アクセス'].apply(lambda x: np.array(
    [z for z in [y.split("\t") for y in x.split("\t\t")] if len(z) == 3]))

station_names = set()

for access_lists in train['アクセス'].dropna():
    for access_list in access_lists:
        station_names.add(access_list[1])

for station_name in station_names:
    times = []
    for access_lists in train['アクセス']:
        times.append(0)
        if type(access_lists) == float:
            continue
        for access_list in access_lists:
            if station_name == access_list[1]:
                times[len(times)-1] = access_list[2]
    train[station_name] = times

for station_name in station_names:
    train[station_name] = train[station_name].apply(lambda x: str(x).replace("分", "")).apply(lambda x: int(x.split("(")[1].split(")")[0])*0.25 + int(x.split("下車徒歩")[1])*0.08 if "バス" in str(x) and "徒歩" in str(x) else x).apply(
        lambda x: int(x.split("(")[1].split(")")[0])*0.25 if "バス" in str(x) else x).apply(lambda x: float(x.split("車")[1].split("km")[0]) if "車" in str(x) else x).apply(lambda x: int(x.split("徒歩")[1])*0.08 if "徒歩" in str(x) else x)
    train[station_name] = train[station_name].astype(float)


In [10]:
# 間取り


In [11]:
# 築年数
train['築年数'] = train['築年数'].apply(lambda x: x.replace("新築", "0年"))
train['年'] = train['築年数'].apply(lambda x: int(x.split("年")[0]))
train['月'] = train['築年数'].apply(lambda x: int(
    x.split("年")[1].split("ヶ月")[0])/12 if "ヶ月" in x else 0)
train['築年数'] = train['年'] + train['月']


In [12]:
# 方角


In [13]:
# 面積
train['面積'] = train['面積'].apply(lambda x: float(x.split("m2")[0]))


In [14]:
# 所在階
train['所在階'] = train['所在階'].dropna().apply(lambda x: x.replace(
    "階建", "").replace("階", "").replace("地下", "-"))

train['階'] = train['所在階'].dropna().apply(
    lambda x: x.split("／")[0] if "／" in x and x.split("／")[0] != "" else pd.np.nan)

train['階数'] = train['所在階'].dropna().apply(lambda x: x.split(
    "／")[1] if "／" in x else x).apply(lambda x: x.split("（")[0])

train['地下'] = train['所在階'].dropna().apply(lambda x:
                                          x.split("（")[1].split("）")[0] if "（" in x else pd.np.nan)


In [15]:
# バス・トイレ
train['バス・トイレ'] = train['バス・トイレ'].dropna().apply(
    lambda x: x.replace("\t／", "／").split('／\t'))


In [16]:
# キッチン
train['キッチン'] = train['キッチン'].dropna().apply(
    lambda x: x.replace("\t／", "／").split('／\t'))


In [17]:
# 放送・通信　
train['放送・通信'] = train['放送・通信'].dropna().apply(
    lambda x: x.replace("\t／", "／").split('／\t'))


In [18]:
# 室内設備
train['室内設備'] = train['室内設備'].dropna().apply(
    lambda x: x.replace("\t／", "／").replace("／\t", "\t").split("\t"))


In [19]:
# 駐車場
train['駐車場'] = train['駐車場'].dropna().apply(lambda x: x.split('\t'))


In [20]:
# 周辺環境
train['周辺環境'] = train['周辺環境'].dropna().apply(lambda x: [y.replace(
    "m", "").replace("【", "").split("】") for y in x.split("\t")])

env_names = set()

for env_lists in train['周辺環境'].dropna():
    for env_list in env_lists:
        env_names.add(env_list[0])

for env_name in env_names:
    distances = []
    for env_lists in train['周辺環境']:
        distances.append(0)
        if type(env_lists) == float:
            continue
        for env_list in env_lists:
            if env_name == env_list[0]:
                if distances[len(distances)-1] == 0 or int(env_list[1]) < distances[len(distances)-1]:
                    distances[len(distances)-1] = int(env_list[1])
    train[env_name] = distances


In [21]:
# 建物構造


In [22]:
# 契約期間
rented = []
for contact in train['契約期間']:
    if "定期貸家" in str(contact):
        rented.append(1)
    else:
        rented.append(0)
train['定期貸家'] = rented

train['契約期間'] = train['契約期間'].dropna().apply(lambda x: (int(x.split("年")[0])+int(x.split("年")[1].split("月")[0])/12) - 2019+8/12 if "まで" in str(x) else x).apply(lambda x: int(x.split("年")[0]) + int(x.split("年")
                                                                                                                                                                                                     [1].split("ヶ月間")[0])/12 if "ヶ月間" in str(x) and "年" in str(x) else x).apply(lambda x: int(x.split("ヶ月間")[0]) if "ヶ月間" in str(x) else x).apply(lambda x: int(x.split("年間")[0]) if "年間" in str(x) else x)


In [23]:
# ダミー変数化
columns = ['鉄道会社', 'バス・トイレ', 'キッチン', '放送・通信', '室内設備']
for column in columns:
    train = pd.concat([train, pd.get_dummies(
        train[column].apply(pd.Series).stack(dropna=False)).sum(level=0)], axis=1)


In [24]:
test.head(1)


,id,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間
0,31471,東京都世田谷区深沢５丁目1-27,東急大井町線\t等々力駅\t徒歩15分\t\t東急田園都市線\t桜新町駅\t徒歩24分\t\...,2LDK,49年0ヶ月,南,50.22m2,8階／8階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t温水洗浄便座／...,ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\t給湯／\tL字キッチン,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\tエレベーター\t公営水...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【コンビニ】 74m\t【スーパー】 215m\t【病院】 313m\t【小学校】 219m...,鉄骨造,NaN


In [25]:
test.columns


Index(['id', '所在地', 'アクセス', '間取り', '築年数', '方角', '面積', '所在階', 'バス・トイレ', 'キッチン',
       '放送・通信', '室内設備', '駐車場', '周辺環境', '建物構造', '契約期間'],
      dtype='object')

In [26]:
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31262 entries, 0 to 31261
Data columns (total 16 columns):
id        31262 non-null int64
所在地       31262 non-null object
アクセス      31262 non-null object
間取り       31262 non-null object
築年数       31262 non-null object
方角        28454 non-null object
面積        31262 non-null object
所在階       31261 non-null object
バス・トイレ    30942 non-null object
キッチン      30627 non-null object
放送・通信     27808 non-null object
室内設備      31189 non-null object
駐車場       25906 non-null object
周辺環境      22021 non-null object
建物構造      31262 non-null object
契約期間      24281 non-null object
dtypes: int64(1), object(15)
memory usage: 3.8+ MB


In [27]:
test.isnull().sum()


id           0
所在地          0
アクセス         0
間取り          0
築年数          0
方角        2808
面積           0
所在階          1
バス・トイレ     320
キッチン       635
放送・通信     3454
室内設備        73
駐車場       5356
周辺環境      9241
建物構造         0
契約期間      6981
dtype: int64

In [28]:
test.describe(include='all')


,id,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間
count,31262.00,31262,31262,31262,31262,28454,31262,31261,30942,30627,27808,31189,25906,22021,31262,24281
unique,nan,11191,24173,37,734,8,5746,1020,353,379,144,6289,3839,17972,11,58
top,nan,東京都世田谷区太子堂２丁目,都営大江戸線\t新江古田駅\t徒歩10分\t\t西武池袋線\t江古田駅\t徒歩20分\t\t...,1K,0年1ヶ月,南,20m2,2階／2階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t24時間換気システム\...,駐車場\t無,【スーパー】 227m\t【スーパー】 442m\t【スーパー】 422m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間
freq,nan,53,66,11731,902,8788,260,2855,4507,5937,2548,1523,3123,42,17305,22497
mean,47101.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,9024.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,31471.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,39286.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,47101.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,54916.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# 所在地
test['所在地'] = test['所在地'].apply(lambda x: x.replace("丁目-", "丁目"))

test['都道府県'] = test['所在地'].apply(
    lambda x: x.split("都")[0]+"都" if "都" in x else pd.np.nan)
test['市区町村'] = test['所在地'].apply(lambda x: x.split(
    "都")[1].split("区")[0]+"区" if "区" in x else pd.np.nan)
test['丁目番地号'] = test['所在地'].apply(lambda x: x.split(
    "区")[1].split("丁目")[0]+"丁目" if "丁目" in x else pd.np.nan)
test['マンション名・部屋番号等'] = test['所在地'].apply(
    lambda x: x.split("丁目")[1] if "丁目" in x and x.split("丁目")[1] != "" else pd.np.nan)


In [30]:
# アクセス
test['鉄道会社'] = test['アクセス'].apply(lambda x: np.array(
    [z for z in [y.split("\t") for y in x.split("\t\t")] if len(z) == 3]).T[0])
test['駅名'] = test['アクセス'].apply(lambda x: np.array(
    [z for z in [y.split("\t") for y in x.split("\t\t")] if len(z) == 3]).T[1])
test['所要時間'] = test['アクセス'].apply(lambda x: np.array(
    [z for z in [y.split("\t") for y in x.split("\t\t")] if len(z) == 3]).T[2])

test['アクセス'] = test['アクセス'].apply(lambda x: np.array(
    [z for z in [y.split("\t") for y in x.split("\t\t")] if len(z) == 3]))

station_names = set()

for access_lists in test['アクセス'].dropna():
    for access_list in access_lists:
        station_names.add(access_list[1])

for station_name in station_names:
    times = []
    for access_lists in test['アクセス']:
        times.append(0)
        if type(access_lists) == float:
            continue
        for access_list in access_lists:
            if station_name == access_list[1]:
                times[len(times)-1] = access_list[2]
    test[station_name] = times

for station_name in station_names:
    test[station_name] = test[station_name].apply(lambda x: str(x).replace("分", "")).apply(lambda x: int(x.split("(")[1].split(")")[0])*0.25 + int(x.split("下車徒歩")[1])*0.08 if "バス" in str(x) and "徒歩" in str(x) else x).apply(
        lambda x: int(x.split("(")[1].split(")")[0])*0.25 if "バス" in str(x) else x).apply(lambda x: float(x.split("車")[1].split("km")[0]) if "車" in str(x) else x).apply(lambda x: int(x.split("徒歩")[1])*0.08 if "徒歩" in str(x) else x)
    test[station_name] = test[station_name].astype(float)


In [31]:
# 間取り


In [32]:
# 築年数
test['築年数'] = test['築年数'].apply(lambda x: x.replace("新築", "0年"))
test['年'] = test['築年数'].apply(lambda x: int(x.split("年")[0]))
test['月'] = test['築年数'].apply(lambda x: int(
    x.split("年")[1].split("ヶ月")[0])/12 if "ヶ月" in x else 0)
test['築年数'] = test['年'] + test['月']


In [33]:
# 方角


In [34]:
# 面積
test['面積'] = test['面積'].apply(lambda x: float(x.split("m2")[0]))


In [35]:
# 所在階
test['所在階'] = test['所在階'].dropna().apply(lambda x: x.replace(
    "階建", "").replace("階", "").replace("地下", "-"))

test['階'] = test['所在階'].dropna().apply(
    lambda x: x.split("／")[0] if "／" in x and x.split("／")[0] != "" else pd.np.nan)

test['階数'] = test['所在階'].dropna().apply(lambda x: x.split(
    "／")[1] if "／" in x else x).apply(lambda x: x.split("（")[0])

test['地下'] = test['所在階'].dropna().apply(lambda x:
                                        x.split("（")[1].split("）")[0] if "（" in x else pd.np.nan)


In [36]:
# バス・トイレ
test['バス・トイレ'] = test['バス・トイレ'].dropna().apply(
    lambda x: x.replace("\t／", "／").split('／\t'))


In [37]:
# キッチン
test['キッチン'] = test['キッチン'].dropna().apply(
    lambda x: x.replace("\t／", "／").split('／\t'))


In [38]:
# 放送・通信　
test['放送・通信'] = test['放送・通信'].dropna().apply(
    lambda x: x.replace("\t／", "／").split('／\t'))


In [39]:
# 室内設備
test['室内設備'] = test['室内設備'].dropna().apply(
    lambda x: x.replace("\t／", "／").replace("／\t", "\t").split("\t"))


In [40]:
# 駐車場
test['駐車場'] = test['駐車場'].dropna().apply(lambda x: x.split('\t'))


In [41]:
# 周辺環境
test['周辺環境'] = test['周辺環境'].dropna().apply(lambda x: [y.replace(
    "m", "").replace("【", "").split("】") for y in x.split("\t")])

env_names = set()

for env_lists in test['周辺環境'].dropna():
    for env_list in env_lists:
        env_names.add(env_list[0])

for env_name in env_names:
    distances = []
    for env_lists in test['周辺環境']:
        distances.append(0)
        if type(env_lists) == float:
            continue
        for env_list in env_lists:
            if env_name == env_list[0]:
                if distances[len(distances)-1] == 0 or int(env_list[1]) < distances[len(distances)-1]:
                    distances[len(distances)-1] = int(env_list[1])
    test[env_name] = distances


In [42]:
# 建物構造


In [43]:
# 契約期間
rented = []
for contact in test['契約期間']:
    if "定期貸家" in str(contact):
        rented.append(1)
    else:
        rented.append(0)
test['定期貸家'] = rented

test['契約期間'] = test['契約期間'].dropna().apply(lambda x: (int(x.split("年")[0])+int(x.split("年")[1].split("月")[0])/12) - 2019+8/12 if "まで" in str(x) else x).apply(lambda x: int(x.split("年")[0]) + int(x.split("年")
                                                                                                                                                                                                   [1].split("ヶ月間")[0])/12 if "ヶ月間" in str(x) and "年" in str(x) else x).apply(lambda x: int(x.split("ヶ月間")[0]) if "ヶ月間" in str(x) else x).apply(lambda x: int(x.split("年間")[0]) if "年間" in str(x) else x)


In [44]:
# ダミー変数化
columns = ['鉄道会社', 'バス・トイレ', 'キッチン', '放送・通信', '室内設備']
for column in columns:
    test = pd.concat([test, pd.get_dummies(
        test[column].apply(pd.Series).stack(dropna=False)).sum(level=0)], axis=1)


In [45]:
# 列数を揃える
temp1 = list(set(list(test.columns)) - set(list(train.columns)))
temp2 = list(set(list(train.columns)) - set(list(test.columns)))

for column in temp1:
    train[column] = 0

for column in temp2:
    test[column] = 0

test = test.drop('賃料', axis=1)


In [46]:
# エンコーディング
encode_list = ['間取り', '方角', '建物構造', '市区町村', '丁目番地号']
for encode in encode_list:
    encoded = train.groupby(encode)['賃料'].mean()
    train[encode] = train[encode].replace(encoded)
    test[encode] = test[encode].replace(encoded)


In [47]:
# 欠損値処理
train['丁目番地号'] = train['丁目番地号'].astype(float)
train['方角'] = train['方角'].astype(float)
train['階'] = train['階'].astype(float)
train['階数'] = train['階数'].astype(float)
train['契約期間'] = train['契約期間'].astype(float)
train = train.fillna(
    {'丁目番地号': train['丁目番地号'].mean(), '方角': train['方角'].mean(), '階': train['階'].mean(), '階数': train['階数'].mean(), '地下': 0, '契約期間': train['契約期間'].mean()})
train['階'] = train['階'].astype(int)
train['階数'] = train['階数'].astype(int)
train['地下'] = train['地下'].astype(int)


In [48]:
# 欠損値処理
test['丁目番地号'] = test['丁目番地号'].apply(lambda x: np.nan if type(x) == str else x)
test['丁目番地号'] = test['丁目番地号'].astype(float)
test['間取り'] = test['間取り'].apply(lambda x: np.nan if type(x) == str else x)
test['方角'] = test['方角'].astype(float)
test['階'] = test['階'].astype(float)
test['階数'] = test['階数'].astype(float)
test['契約期間'] = test['契約期間'].astype(float)
test['建物構造'] = test['建物構造'].apply(lambda x: np.nan if type(x) == str else x)
test = test.fillna(
    {'丁目番地号': test['丁目番地号'].mean(), '間取り': test['間取り'].mean(), '方角': test['方角'].mean(), '階': test['階'].mean(), '階数': test['階数'].mean(), '地下': 0, '契約期間': test['契約期間'].mean(), '建物構造': test['建物構造'].mean()})
test['階'] = test['階'].astype(int)
test['階数'] = test['階数'].astype(int)
test['地下'] = test['地下'].astype(int)


In [49]:
# 訓練データ・テストデータ作成
#columns = ['築年数', '面積', '階', '階数', '地下']
remove_list = ['id', '賃料', '所在地', 'アクセス', '所在階', 'バス・トイレ', 'キッチン', '放送・通信',
               '室内設備', '駐車場', '周辺環境', '都道府県', 'マンション名・部屋番号等', '鉄道会社', '駅名', '所要時間', '年', '月']
columns = train.columns.tolist()
[columns.remove(temp) for temp in remove_list]
X_train = train[columns]
y_train = train['賃料']


In [50]:
# 訓練データ・テストデータ作成
#columns = ['築年数', '面積', '階', '階数', '地下']
remove_list = ['id', '所在地', 'アクセス', '所在階', 'バス・トイレ', 'キッチン', '放送・通信',
               '室内設備', '駐車場', '周辺環境', '都道府県', 'マンション名・部屋番号等', '鉄道会社', '駅名', '所要時間', '年', '月']
columns = test.columns.tolist()
[columns.remove(temp) for temp in remove_list]
X_test = test[columns]


In [51]:
# スコア確認
'''
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(
    X_train, y_train, random_state=0)

sc = StandardScaler()
sc.fit(X_train_train)
X_train_train_std = sc.transform(X_train_train)
X_train_test_std = sc.transform(X_train_test)

models = [
    LinearRegression(),
    # LogisticRegression(),
    DecisionTreeRegressor(),
    DecisionTreeClassifier(),
    LinearSVR(),
    # LinearSVC(),
    RandomForestRegressor(),
    # RandomForestClassifier(),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=5),
    GradientBoostingRegressor(),
    # GradientBoostingClassifier(),
    AdaBoostRegressor(DecisionTreeRegressor(), random_state=0),
    # AdaBoostClassifier(n_estimators=50, learning_rate=1.0),
    Ridge(),
    # SVC(gamma=0.001, C=1),
    # OneVsRestClassifier(SVC(kernel='linear', probability=True, random_state=0)),
    # BaggingClassifier(KNeighborsClassifier(), n_estimators=100, random_state=0)
]

scores = {}
for model in models:
    model.fit(X_train_train_std, y_train_train)
    scores[(model.__class__.__name__, 'train_score')
           ] = model.score(X_train_train_std, y_train_train)
    scores[(model.__class__.__name__, 'test_score')
           ] = model.score(X_train_test_std, y_train_test)
pd.Series(scores).unstack()
'''


"\nX_train_train, X_train_test, y_train_train, y_train_test = train_test_split(\n    X_train, y_train, random_state=0)\n\nsc = StandardScaler()\nsc.fit(X_train_train)\nX_train_train_std = sc.transform(X_train_train)\nX_train_test_std = sc.transform(X_train_test)\n\nmodels = [\n    LinearRegression(),\n    # LogisticRegression(),\n    DecisionTreeRegressor(),\n    DecisionTreeClassifier(),\n    LinearSVR(),\n    # LinearSVC(),\n    RandomForestRegressor(),\n    # RandomForestClassifier(),\n    KNeighborsRegressor(n_neighbors=5),\n    KNeighborsClassifier(n_neighbors=5),\n    GradientBoostingRegressor(),\n    # GradientBoostingClassifier(),\n    AdaBoostRegressor(DecisionTreeRegressor(), random_state=0),\n    # AdaBoostClassifier(n_estimators=50, learning_rate=1.0),\n    Ridge(),\n    # SVC(gamma=0.001, C=1),\n    # OneVsRestClassifier(SVC(kernel='linear', probability=True, random_state=0)),\n    # BaggingClassifier(KNeighborsClassifier(), n_estimators=100, random_state=0)\n]\n\nscores =

In [52]:
# グリッドサーチ
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(
    X_train, y_train, random_state=0)

param_grid = {'max_depth': [2, 3, 4, 5], 'min_samples_leaf': [2, 3, 4, 5]}
model = RandomForestRegressor()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train_train, y_train_train)

scores = {}
scores[(model.__class__.__name__, 'train_score')
       ] = grid_search.score(X_train_train, y_train_train)
scores[(model.__class__.__name__, 'test_score')
       ] = grid_search.score(X_train_test, y_train_test)
scores[(model.__class__.__name__, 'train_RMSE')] = np.sqrt(mean_squared_error(
    y_train_train, grid_search.predict(X_train_train)))
scores[(model.__class__.__name__, 'test_RMSE')] = np.sqrt(mean_squared_error(
    y_train_test, grid_search.predict(X_train_test)))

print('スコアがベストなときのパラメータ:{}'.format(grid_search.best_params_))
print('スコアがベストなときのross-validation score:{:2f}'.format(grid_search.best_score_))

pd.Series(scores).unstack()


スコアがベストなときのパラメータ:{'max_depth': 5, 'min_samples_leaf': 2}
スコアがベストなときのross-validation score:0.838930


,test_RMSE,test_score,train_RMSE,train_score
RandomForestRegressor,31906.30,0.81,26641.25,0.87


In [53]:
# スコア確認(ランダムフォレスト)
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(
    X_train, y_train, random_state=0)

#model = RandomForestRegressor()
model = LinearRegression()

scores = {}
model.fit(X_train_train, y_train_train)
scores[(model.__class__.__name__, 'train_score')
       ] = model.score(X_train_train, y_train_train)
scores[(model.__class__.__name__, 'test_score')
       ] = model.score(X_train_test, y_train_test)
scores[(model.__class__.__name__, 'train_RMSE')] = np.sqrt(mean_squared_error(
    y_train_train, model.predict(X_train_train)))
scores[(model.__class__.__name__, 'test_RMSE')] = np.sqrt(mean_squared_error(
    y_train_test, model.predict(X_train_test)))

pd.Series(scores).unstack()


,test_RMSE,test_score,train_RMSE,train_score
LinearRegression,33987.41,0.79,29095.19,0.84


In [54]:
# sc = StandardScaler()
# sc.fit(X_train)
# X_train_std = sc.transform(X_train)
# X_test_std = sc.transform(X_test)


In [55]:
# モデルに適用
scores = {}
#model = RandomForestRegressor()
model = LinearRegression()
model.fit(X_train, y_train)
scores[(model.__class__.__name__, 'train_score')
       ] = model.score(X_train, y_train)
pd.Series(scores).unstack()


,train_score
LinearRegression,0.83


In [56]:
# 提出ファイル書き出し
pred_y = model.predict(X_test)
sample_submit[1] = pred_y
sample_submit.to_csv('predict1.csv', index=None, header=None)


In [57]:
# csv書き出し
'''
single_columns = {"都道府県": "state", "市区町村": "city", "丁目番地号": "address1", "マンション名・部屋番号等": "address2", "間取り": "layout", "築年数": "age",
                  "方角": "direction", "面積": "area", "建物構造": "architecture", "契約期間": "contract", "階": "floor", "階数": "story", "地下": "basement"}
list_columns = {"鉄道会社": "company", "駅名": "station",
                "所要時間": "time", "バス・トイレ": "bath_toilet", "キッチン": "kitchen", "放送・通信": "communication", "室内設備": "equipment", "駐車場": "parking"}

for column, file_name in single_columns.items():
    pd.Series(train[column].dropna().unique()).sort_values().to_csv(
        file_name + ".csv", index=None)
    train[column].dropna().value_counts().to_csv(file_name + "_ranking.csv")
for column, file_name in list_columns.items():
    pd.Series(train[column].dropna().apply(pd.Series).stack(
    ).unique()).sort_values().to_csv(file_name + ".csv", index=None)
    train[column].dropna().apply(pd.Series).stack(
    ).value_counts().to_csv(file_name + "_ranking.csv")
'''


'\nsingle_columns = {"都道府県": "state", "市区町村": "city", "丁目番地号": "address1", "マンション名・部屋番号等": "address2", "間取り": "layout", "築年数": "age",\n                  "方角": "direction", "面積": "area", "建物構造": "architecture", "契約期間": "contract", "階": "floor", "階数": "story", "地下": "basement"}\nlist_columns = {"鉄道会社": "company", "駅名": "station",\n                "所要時間": "time", "バス・トイレ": "bath_toilet", "キッチン": "kitchen", "放送・通信": "communication", "室内設備": "equipment", "駐車場": "parking"}\n\nfor column, file_name in single_columns.items():\n    pd.Series(train[column].dropna().unique()).sort_values().to_csv(\n        file_name + ".csv", index=None)\n    train[column].dropna().value_counts().to_csv(file_name + "_ranking.csv")\nfor column, file_name in list_columns.items():\n    pd.Series(train[column].dropna().apply(pd.Series).stack(\n    ).unique()).sort_values().to_csv(file_name + ".csv", index=None)\n    train[column].dropna().apply(pd.Series).stack(\n    ).value_counts().to_csv(file_name + "_ranking.csv"